In [1]:
import aircv as ac
import numpy as np

In [2]:
from pathlib import Path
from PIL import Image
from fplab.tools.image import get_data_dirs
from fplab.tools.array import IMA

### delete files

In [ ]:
script_d = r"E:\Projects\fingerprint\data\sources\MOLF\DB4_unrename_dirs.txt"
root_d = r"E:\Projects\fingerprint\data\sources\MOLF\DB4_Latent_renamed"
img_dirs = []
img_dirs = get_data_dirs(root_d, img_dirs)

In [ ]:
remove_dirs = []
remove_flag = ["_1_1.png", "_1_2.png", "_5_1.png", "_5_2.png"]
for d in img_dirs:
    for flag in remove_flag:
        if flag in d:
            remove_dirs.append(d)

In [ ]:
with open(script_d, "a") as f:
    f.write("remove_dirs:\n")
    for d in remove_dirs:
        f.write(f"{d}\n")
        im_p = Path(d)
        im = Image.open(im_p)
        im.save(d.replace("DB4_Latent_renamed", "DB4_Latent_unrenamed"))
        im_p.unlink()

### rename files

In [3]:
script_d = r"E:\Projects\fingerprint\data\sources\MOLF\DB4_rename_scripts.txt"
root_d = r"E:\Projects\fingerprint\data\sources\MOLF\DB4_Latent_renamed"
root_source_d = root_d.replace("DB4_Latent_renamed", "DB5_SimLatent")
img_dirs = []
img_dirs = get_data_dirs(root_d, img_dirs)

In [4]:
img_infos = {}
i = 0
for d in img_dirs:
    im_p = Path(d)
    stems = im_p.stem.split("_")
    source = f"{stems[0]}_{stems[1]}_{stems[2]}.png"
    source_d = Path(root_source_d)/source
    im_source = IMA.read(source_d).l2rgb().ima
    im_search = IMA.read(d).l2rgb().ima
    x, y = ac.find_template(im_source, im_search)['result']
    img_infos[d]=x

In [5]:
for d in img_dirs:
    info = img_infos[d]
    im_p = Path(d)
    im = Image.open(im_p)
    stems = im_p.stem.split("_")
    old_name = f"{stems[0]}_{stems[1]}_{stems[2]}_{stems[3]}.png"
    if stems[2] in ["2", "6"]:
        infos = []
        for i in range(2):
            infos.append(img_infos[d.replace(f"_{stems[3]}.png", f"_{i+1}.png")])
        ind = sorted(infos).index(info)
        if stems[1] == "L":
            finger_id = 8-ind
        else:
            assert stems[1] == "R"
            finger_id = 2+ind
        new_name = f"{stems[0]}_{stems[2]}_{finger_id}.png"
    if stems[2] in ["3", "7"]:
        infos = []
        for i in range(3):
            infos.append(img_infos[d.replace(f"_{stems[3]}.png", f"_{i+1}.png")])
        ind = sorted(infos).index(info)
        if stems[1] == "L":
            finger_id = 9-ind
        else:
            assert stems[1] == "R"
            finger_id = 2+ind
        new_name = f"{stems[0]}_{stems[2]}_{finger_id}.png"
    if stems[2] in ["4", "8"]:
        infos = []
        for i in range(4):
            infos.append(img_infos[d.replace(f"_{stems[3]}.png", f"_{i+1}.png")])
        ind = sorted(infos).index(info)
        if stems[1] == "L":
            finger_id = 10-ind
        else:
            assert stems[1] == "R"
            finger_id = 2+ind
        new_name = f"{stems[0]}_{stems[2]}_{finger_id}.png"     
    with open(script_d, "a") as f:
        f.write(f"{old_name}->{new_name}\n")
    im.save(d.replace(old_name, new_name))
    im_p.unlink()